In [22]:
import igl
import meshio as mio
import numpy as np
import math
import sys
import scipy

In [24]:
init_file = "icosphere_initial_tetmesh.msh"
solution_file = "solution.txt"
target_file = "CT/laplace_beltrami_mesh.msh"
local2global_file = "icosphere_tri_to_tet_v_map.txt"
b2l_mat_file = "CT_bezier_to_lag_convertion_matrix.txt"
output_name = "test_deviation"

In [8]:
# read initial mesh and solution
init_mesh = mio.read(init_file)
v_init = init_mesh.points
t_init = init_mesh.cells_dict['tetra20']

solution = np.loadtxt(solution_file)
local2global = np.loadtxt(local2global_file).astype(np.int32)

In [9]:
# get result beizer control points
v_res = v_init + solution

In [10]:
v_res_sf = v_res[local2global]

In [25]:
# convert to lagrange
b2l_mat = scipy.io.mmread(b2l_mat_file)

In [26]:
v_res_sf_lag = b2l_mat @ v_res_sf

In [11]:
# read target mesh
target_mesh = mio.read(target_file)
v_target = target_mesh.points

In [27]:
# compute deviation
deviation = v_res_sf_lag - v_target

In [36]:
distance = np.array([np.linalg.norm(d) for d in deviation])

In [41]:
max_distance = np.max(distance)
avg_distance = np.average(distance)

In [44]:
# write to file
with open(output_name + "_max_avg_deviation.txt", "w") as file:
    file.write("{}\n".format(max_distance))
    file.write("{}\n".format(avg_distance))

In [35]:
avg_deviation

3.496450526688195e-15

In [33]:
max_deviation

2.7644553313166398e-14

In [30]:
np.max(deviation)

2.1371793224034263e-14

In [31]:
np.min(deviation)

-2.7644553313166398e-14

In [28]:
test_mesh = mio.Mesh(v_res_sf_lag, [("triangle10", target_mesh.cells_dict['triangle10'])])

In [29]:
test_mesh.write("test_deviation.msh", file_format="gmsh")

[[ 1.88737914e-15  3.66373598e-15  3.16413562e-15]
 [ 1.88737914e-15  4.21884749e-15 -6.02295991e-15]
 [-8.32667268e-15 -3.33066907e-16  1.59941504e-15]
 ...
 [-1.87322987e-03 -6.53788285e-04 -4.17143791e-04]
 [-2.24558374e-03  2.60792234e-03  2.63412344e-04]
 [-4.55191440e-15 -4.53803661e-15  3.33066907e-16]]
